In [ ]:
!pip install -U "transformers" "huggingface_hub" "tokenizers"
!pip install Levenshtein

In [ ]:
import json
import pandas as pd
import numpy as np
import json
import pandas as pd
import numpy as np
import os, math, pandas as pd, numpy as np
from dataclasses import dataclass
from typing import Dict, List, Union, Optional
import torch
import Levenshtein
from typing import List, Sequence, Tuple
from transformers import (AutoTokenizer, AutoConfig, AutoModelForSequenceClassification,
                          TrainingArguments, Trainer, DataCollatorWithPadding)
from sklearn.metrics import accuracy_score, f1_score
from scipy.stats import spearmanr, pearsonr
import editdistance
from peft import LoraConfig, get_peft_model, TaskType
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn.functional as F
from torch.cuda.amp import autocast, GradScaler
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModel, AutoConfig
from tqdm import tqdm
import os
import random
import torch
from transformers import AutoTokenizer, AutoModel, AutoConfig
from peft import PeftModel
from collections import deque
from tqdm import tqdm
import os
import torch
import torch.optim as optim
import numpy as np
from transformers import get_scheduler
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from torch.cuda.amp import autocast, GradScaler
import kagglehub
import random
from scipy.stats import spearmanr

from types import SimpleNamespace

In [ ]:
TASK_TYPE = "pair_cls"
MAX_LENGTH = 256
BATCH_SIZE = 2
epochs = 5
LR = 2e-5
warmup_ratio = 0.06
save_dir = 'ckpt1'
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

###
blending_model_special_token = "_" # teacher
base_model_special_token = "##" # student

In [ ]:


class TextPairRaw(Dataset):
    def __init__(self, df: pd.DataFrame, task: str):
        self.task = task
        if task == "single_cls":
            self.samples = [(t, None, int(y)) for t, y in zip(df["text"].astype(str), df["label"].astype(int))]
        elif task == "pair_cls":
            self.samples = [(a, b) for a,b in zip(df["premise"].astype(str),
                                                            df["hypothesis"].astype(str))]
        else:  # pair_reg
            self.samples = [(a, b) for a,b in zip(df["sentence1"].astype(str),
                                                              df["sentence2"].astype(str))]
    def __len__(self): return len(self.samples)
    def __getitem__(self, idx): return self.samples[idx] 
from typing import List, Tuple, Optional



class DualTokenizerCollate:
    def __init__(self, tok_student, tok_teacher, task: str, max_len: int):
        self.ts = tok_student
        self.tt = tok_teacher
        self.task = task
        self.max_len = max_len

    def __call__(self, batch: List[Tuple[str, Optional[str], float]]):
        s1s, s2s = zip(*batch)

        if self.task == "single_cls":
            s_enc = self.ts(list(s1s), max_length=self.max_len, truncation=True,
                            padding=True, return_tensors="pt",
                            return_special_tokens_mask=True)
            t_enc = self.tt(list(s1s), max_length=self.max_len, truncation=True,
                            padding=True, return_tensors="pt",
                            return_special_tokens_mask=True)

            out = {
                "input_ids_stu": s_enc["input_ids"],
                "attention_mask_stu": s_enc["attention_mask"],
                "special_tokens_mask_stu": s_enc["special_tokens_mask"],
                "input_ids_tea": t_enc["input_ids"],
                "attention_mask_tea": t_enc["attention_mask"],
                "special_tokens_mask_tea": t_enc["special_tokens_mask"],
                "labels": torch.tensor(ys, dtype=torch.long),
            }
            if "token_type_ids" in s_enc:
                out["token_type_ids_stu"] = s_enc["token_type_ids"]
            if "token_type_ids" in t_enc:
                out["token_type_ids_tea"] = t_enc["token_type_ids"]
            return out

        # ------- pair (bi-encoder) -------
        s1_enc = self.ts(list(s1s), max_length=self.max_len, truncation=True,
                         padding=True, return_tensors="pt",
                         return_special_tokens_mask=True)
        s2_enc = self.ts(list(s2s), max_length=self.max_len, truncation=True,
                         padding=True, return_tensors="pt",
                         return_special_tokens_mask=True)

        t1_enc = self.tt(list(s1s), max_length=self.max_len, truncation=True,
                         padding=True, return_tensors="pt",
                         return_special_tokens_mask=True)
        t2_enc = self.tt(list(s2s), max_length=self.max_len, truncation=True,
                         padding=True, return_tensors="pt",
                         return_special_tokens_mask=True)

        out = {
            # student
            "input_ids1_stu": s1_enc["input_ids"],
            "attention_mask1_stu": s1_enc["attention_mask"],
            "special_tokens_mask1_stu": s1_enc["special_tokens_mask"],
            "input_ids2_stu": s2_enc["input_ids"],
            "attention_mask2_stu": s2_enc["attention_mask"],
            "special_tokens_mask2_stu": s2_enc["special_tokens_mask"],
            # teacher
            "input_ids1_tea": t1_enc["input_ids"],
            "attention_mask1_tea": t1_enc["attention_mask"],
            "special_tokens_mask1_tea": t1_enc["special_tokens_mask"],
            "input_ids2_tea": t2_enc["input_ids"],
            "attention_mask2_tea": t2_enc["attention_mask"],
            "special_tokens_mask2_tea": t2_enc["special_tokens_mask"],
        }
        # chỉ thêm token_type_ids nếu tồn tại
        if "token_type_ids" in s1_enc:
            out["token_type_ids1_stu"] = s1_enc["token_type_ids"]
        if "token_type_ids" in s2_enc:
            out["token_type_ids2_stu"] = s2_enc["token_type_ids"]
        if "token_type_ids" in t1_enc:
            out["token_type_ids1_tea"] = t1_enc["token_type_ids"]
        if "token_type_ids" in t2_enc:
            out["token_type_ids2_tea"] = t2_enc["token_type_ids"]

        return out


print("Done Prepare Dataset")

In [ ]:
def info_nce(q, k, temperature=0.07, neg_valid_mask=None):
    q = F.normalize(q, dim=-1)
    k = F.normalize(k, dim=-1)

    logits = torch.matmul(q, k.T) / temperature
    labels = torch.arange(q.size(0), device=q.device)
    loss_inbatch = F.cross_entropy(logits, labels) 
    return loss_inbatch, logits

In [ ]:
data_train = '/kaggle/input/final-data/final_data.csv'
df = pd.read_csv(data_train)
df.head(3)

In [ ]:
df["premise"] = df["text"]
df["hypothesis"] = df["text"]
cols = ["premise", "hypothesis"]  # hoặc ["premise","hypothesis","label"] nếu cần placeholder
df_out = df[cols].copy()

In [ ]:
tok_student = AutoTokenizer.from_pretrained("BAAI/bge-m3")
model_student = AutoModel.from_pretrained(
    "BAAI/bge-m3",
    output_hidden_states=True
)
tok_teacher = AutoTokenizer.from_pretrained("BAAI/bge-m3")
model_teacher = AutoModel.from_pretrained(
    "BAAI/bge-m3",
    output_hidden_states=True
)

In [ ]:

train_ds = TextPairRaw(df_out, TASK_TYPE)

collate = DualTokenizerCollate(tok_student, tok_teacher, TASK_TYPE, MAX_LENGTH)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True,
                          collate_fn=collate, pin_memory=True, num_workers=2, persistent_workers=True)
print("Done Prepare Dataloader")

In [ ]:
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from tqdm import tqdm
import torch.nn.functional as F
from scipy.stats import pearsonr, spearmanr

tokenizer = AutoTokenizer.from_pretrained('BAAI/bge-m3')

class STSDataset(Dataset):
    def __init__(self, file_path):
        self.dataset = pd.read_csv(file_path)

    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, idx):
        # instruction = "Given a text, Retrieve semantically similar text: "
        instruction=""
        return {
            "sentence1": instruction + self.dataset.iloc[idx]['sentence1'],
            "sentence2": instruction + self.dataset.iloc[idx]['sentence2'],
            "label": torch.tensor(self.dataset.iloc[idx]['score'], dtype=torch.float),
        }
        
def collate_fn(batch, tokenizer, max_len=128):
    s1_list = [item["sentence1"] for item in batch]
    s2_list = [item["sentence2"] for item in batch]
    labels = torch.stack([item["label"] for item in batch])

    enc1 = tokenizer(
        s1_list,
        truncation=True,
        padding=True,       # chỉ pad theo câu dài nhất trong batch
        max_length=max_len,
        return_tensors="pt"
    )
    enc2 = tokenizer(
        s2_list,
        truncation=True,
        padding=True,
        max_length=max_len,
        return_tensors="pt"
    )

    return {
        "input_ids1": enc1["input_ids"],
        "attention_mask1": enc1["attention_mask"],
        "input_ids2": enc2["input_ids"],
        "attention_mask2": enc2["attention_mask"],
        "labels": labels,
    }

# Mean pooling function

def mean_pooling(model_output, attention_mask):
    """
    Mean pooling - take attention mask into account for correct averaging
    """
    token_embeddings = model_output.last_hidden_state  # [B, L, D]
    
    # Expand attention mask to match token embeddings shape
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    
    # Sum embeddings along sequence length, weighted by attention mask
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, dim=1)  # [B, D]
    
    # Sum of attention mask for normalization
    sum_mask = torch.clamp(input_mask_expanded.sum(dim=1), min=1e-9)  # [B, D]
    
    # Average
    mean_embeddings = sum_embeddings / sum_mask  # [B, D]
    
    return mean_embeddings


# Modified eval_sts function with Matryoshka dimensions
def eval_sts(model, eval_loader, matryoshka_dims=[16, 32, 64, 128, 256, 512, 1024]):
    results = {}
    device = model.device

    for dim in matryoshka_dims:
        preds, labels = [], []
        
        with torch.cuda.amp.autocast(dtype=torch.float16):
            with torch.no_grad():
                for batch in tqdm(eval_loader, desc=f"Eval dim={dim}"):
                    input_ids1 = batch["input_ids1"].to(device)
                    attn1 = batch["attention_mask1"].to(device)
                    input_ids2 = batch["input_ids2"].to(device)
                    attn2 = batch["attention_mask2"].to(device)
                    label = batch["labels"]

                    out1 = model(input_ids=input_ids1, attention_mask=attn1)
                    out2 = model(input_ids=input_ids2, attention_mask=attn2)

                    # emb1 = mean_pooling(out1, attn1)[:, :dim]  # Slice to matryoshka dim
                    # emb2 = mean_pooling(out2, attn2)[:, :dim]  # Slice to matryoshka dim
                    emb1 = out1.last_hidden_state[:, 0, :][:, :dim]
                    emb2 = out2.last_hidden_state[:, 0, :][:, :dim]

                    # cosine similarity
                    sim = F.cosine_similarity(emb1, emb2)
                    score = (sim + 1) * 2.5  # scale [-1,1] -> [0,5]

                    preds.extend(score.cpu().numpy())
                    labels.extend(label.numpy())

        spearman_corr, _ = spearmanr(preds, labels)
        results[f"dim_{dim}"] = spearman_corr
        print(f"  Dim {dim}: Spearman = {spearman_corr:.4f}")

    return results


# Modified eval_sts_task
def eval_sts_task(model, path_list):
    model.eval()
    print(' ✅ eval_sts_task')
    for path in path_list:
        print(f"\n{path}")
        eval_dataset = STSDataset(path)
        eval_loader = DataLoader(
            eval_dataset,
            batch_size=64,
            shuffle=False,
            collate_fn=lambda x: collate_fn(x, tokenizer)
        )
        results = eval_sts(model, eval_loader)
    model.train()

In [ ]:
from sklearn.metrics import accuracy_score, f1_score
from sklearn.linear_model import LogisticRegression
import datasets
import numpy as np
import torch

# Modified eval_cls function with Matryoshka dimensions
def eval_cls(model, eval_loader, dim=1024):
    preds, labels = [], []
    device = model.device

    with torch.cuda.amp.autocast(dtype=torch.float16):
        with torch.no_grad():
            for batch in tqdm(eval_loader, desc=f"Extract dim={dim}"):
                input_ids1 = batch["input_ids1"].to(device)
                attn1 = batch["attention_mask1"].to(device)
                label = batch["labels"]

                out1 = model(input_ids=input_ids1, attention_mask=attn1)
                #emb1 = mean_pooling(out1, attn1)[:, :dim]  # Slice to matryoshka dim
                emb1 = out1.last_hidden_state[:, 0, :][:, :dim]

                preds.extend(emb1.cpu().numpy())
                labels.extend(label.numpy())

    return preds, labels





class ClasssifyDataset(Dataset):
    def __init__(self, file_path):
        self.dataset = pd.read_csv(file_path)

    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, idx):
        return {
            "text": self.dataset.iloc[idx]['text'],
            "label": torch.tensor(self.dataset.iloc[idx]['label'], dtype=torch.long),
        }

def clf_collate_fn(batch, tokenizer, max_len=512):
    s1_list = [item["text"] for item in batch]
    labels = torch.stack([item["label"] for item in batch])

    enc1 = tokenizer(
        s1_list,
        truncation=True,
        padding=True,       # chỉ pad theo câu dài nhất trong batch
        max_length=max_len,
        return_tensors="pt"
    )

    return {
        "input_ids1": enc1["input_ids"],
        "attention_mask1": enc1["attention_mask"],
        "labels": labels,
    }


# Modified eval_classification_task
def eval_classification_task(model, path_list):
    model.eval()
    print(' ✅ eval classifier')
    matryoshka_dims = [16, 32, 64, 128, 256, 512, 1024]

    for train_path, dev_path in path_list:
        print(f"\n{dev_path}")
        
        for dim in matryoshka_dims:
            eval_dataset = ClasssifyDataset(dev_path)
            eval_loader = DataLoader(
                eval_dataset,
                batch_size=64,
                shuffle=False,
                collate_fn=lambda x: clf_collate_fn(x, tokenizer)
            )

            train_dataset = ClasssifyDataset(train_path)
            train_loader = DataLoader(
                train_dataset,
                batch_size=64,
                shuffle=False,
                collate_fn=lambda x: clf_collate_fn(x, tokenizer)
            )

            X_train, y_train = eval_cls(model, train_loader, dim=dim)
            X_test, y_test = eval_cls(model, eval_loader, dim=dim)

            clf = LogisticRegression(
                random_state=42,
                n_jobs=1,
                max_iter=200,
                verbose=0,
            )
            clf.fit(X_train, y_train)
            y_pred = clf.predict(X_test)

            accuracy = accuracy_score(y_test, y_pred)
            f1 = f1_score(y_test, y_pred, average="macro")
            print(f"  Dim {dim}: Accuracy = {accuracy:.4f}, F1 = {f1:.4f}")

    model.train()
        


In [ ]:
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from tqdm import tqdm
import torch.nn.functional as F
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, average_precision_score


tokenizer = AutoTokenizer.from_pretrained('BAAI/bge-m3')

class PairDataset(Dataset):
    def __init__(self, file_path):
        self.dataset = pd.read_csv(file_path)

    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, idx):
        # instruction = "Given a text, Retrieve semantically similar text: "
        instruction=""
        return {
            "sentence1": instruction + self.dataset.iloc[idx]['sentence1'],
            "sentence2": instruction + self.dataset.iloc[idx]['sentence2'],
            "label": torch.tensor(self.dataset.iloc[idx]['label'], dtype=torch.float),
        }
        

# Modified eval_pair function with Matryoshka dimensions
def eval_pair(model, eval_loader, matryoshka_dims=[16, 32, 64, 128, 256, 512, 1024]):
    results = {}
    device = model.device

    for dim in matryoshka_dims:
        preds, labels = [], []
        
        with torch.cuda.amp.autocast(dtype=torch.float16):
            with torch.no_grad():
                for batch in tqdm(eval_loader, desc=f"Eval dim={dim}"):
                    input_ids1 = batch["input_ids1"].to(device)
                    attn1 = batch["attention_mask1"].to(device)
                    input_ids2 = batch["input_ids2"].to(device)
                    attn2 = batch["attention_mask2"].to(device)
                    label = batch["labels"]

                    out1 = model(input_ids=input_ids1, attention_mask=attn1)
                    out2 = model(input_ids=input_ids2, attention_mask=attn2)

                    # emb1 = mean_pooling(out1, attn1)[:, :dim]  # Slice to matryoshka dim
                    # emb2 = mean_pooling(out2, attn2)[:, :dim]  # Slice to matryoshka dim

                    emb1 = out1.last_hidden_state[:, 0, :][:, :dim]
                    emb2 = out2.last_hidden_state[:, 0, :][:, :dim]

                    # cosine similarity
                    sim = F.cosine_similarity(emb1, emb2)
                    score = (sim + 1) / 2

                    preds.extend(score.cpu().numpy())
                    labels.extend(label.numpy())

        metric = get_metric_pair_classification(preds, labels)
        results[f"dim_{dim}"] = metric
        print(f"  Dim {dim}: {metric}")

    return results
    
    metric = get_metric_pair_classification(preds, labels)
    print(metric)

    return metric

def get_metric_pair_classification(scores, labels):
    best_acc, best_thr = 0, 0
    for thr in np.linspace(0, 1, 200):
        preds = (scores >= thr).astype(int)
        acc = accuracy_score(labels, preds)
        if acc > best_acc:
            best_acc, best_thr = acc, thr
    preds = (scores >= best_thr).astype(int)
    return {
        "best_threshold": best_thr,
        "accuracy": best_acc,
        "f1": f1_score(labels, preds, average="macro"),
        "precision": precision_score(labels, preds, average="macro"),
        "recall": recall_score(labels, preds, average="macro"),
        "average_precision": average_precision_score(labels, scores)
    }


# Modified eval_pair_task
def eval_pair_task(model, path_list):
    model.eval()
    print(' ✅ eval_pair_task')
    for path in path_list:
        print(f"\n{path}")
        eval_dataset = PairDataset(path)
        eval_loader = DataLoader(
            eval_dataset,
            batch_size=64,
            shuffle=False,
            collate_fn=lambda x: collate_fn(x, tokenizer)
        )
        results = eval_pair(model, eval_loader)
    model.train()

In [ ]:
eval_cls_tasks = [('/kaggle/input/multitask-data/multi-data/banking_train.csv', 
                   '/kaggle/input/multitask-data/multi-data/banking77_validation.csv'),
                  ('/kaggle/input/multitask-data/multi-data/emotion_train.csv', 
                   '/kaggle/input/multitask-data/multi-data/emotion_validation.csv'), 
                  ('/kaggle/input/multitask-data/multi-data/tweet_train.csv', 
                   '/kaggle/input/multitask-data/multi-data/tweet_validation.csv')]

eval_sts_tasks = ['/kaggle/input/multitask-data/multi-data/sick_validation.csv', 
                  '/kaggle/input/multitask-data/multi-data/sts12_validation.csv', 
                  '/kaggle/input/multitask-data/multi-data/stsb_validation.csv']

eval_pair_tasks = ['/kaggle/input/multitask-data/multi-data/mrpc_validation.csv', 
                   '/kaggle/input/multitask-data/multi-data/scitail_validation.csv', 
                   '/kaggle/input/multitask-data/multi-data/wic_validation.csv']
test_cls_tasks = [('/kaggle/input/multitask-data/multi-data/banking_train.csv', 
                   '/kaggle/input/multitask-data/multi-data/banking77_test.csv'),
                  ('/kaggle/input/multitask-data/multi-data/emotion_train.csv', 
                   '/kaggle/input/multitask-data/multi-data/emotion_test.csv'), 
                  ('/kaggle/input/multitask-data/multi-data/tweet_train.csv', 
                   '/kaggle/input/multitask-data/multi-data/tweet_test.csv')]

test_sts_tasks = ['/kaggle/input/multitask-data/multi-data/sick_test.csv', 
                  '/kaggle/input/multitask-data/multi-data/sts12_test.csv', 
                  '/kaggle/input/multitask-data/multi-data/stsb_test.csv',
                 '/kaggle/input/sts-data/more_test_data/sick_r.csv', 
                  '/kaggle/input/sts-data/more_test_data/sts13.csv', 
                  '/kaggle/input/sts-data/more_test_data/sts14.csv',
                  '/kaggle/input/sts-data/more_test_data/sts15.csv',
                  '/kaggle/input/sts-data/more_test_data/sts16.csv']

test_pair_tasks = ['/kaggle/input/multitask-data/multi-data/mrpc_test.csv', 
                   '/kaggle/input/multitask-data/multi-data/scitail_test.csv', 
                   '/kaggle/input/multitask-data/multi-data/wic_test.csv']






In [ ]:
best_f1_macro = -1.0
all_preds, all_labels = [], []
total_loss = 0.0

scaler = torch.cuda.amp.GradScaler(enabled=torch.cuda.is_available())
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


num_steps = len(train_loader)
total_traning_steps = num_steps * epochs
warmup_ratio = 0.1

def pick_devices():
    if torch.cuda.device_count() >= 2:
        dev_s = torch.device("cuda:0")  # student
        dev_t = torch.device("cuda:1")  # teacher
    elif torch.cuda.is_available():
        print("[WARN] Only 1 GPU available -> both on cuda:0")
        dev_s = dev_t = torch.device("cuda:0")
    else:
        print("[WARN] No GPU -> CPU")
        dev_s = dev_t = torch.device("cpu")
    return dev_s, dev_t

device_s, device_t = pick_devices()

model_student.to(device_s)
model_teacher.to(device_t)
model_teacher.eval()
for p in model_teacher.parameters():
    p.requires_grad_(False)

proj_s2t = None

optimizer = optim.AdamW(model_student.parameters(), lr=LR)
scaler = GradScaler(enabled=torch.cuda.is_available())
scheduler = get_scheduler(
    name='cosine_with_min_lr',
    optimizer=optimizer,
    num_warmup_steps=int(total_traning_steps * warmup_ratio),
    # num_warmup_steps=1,
    num_training_steps=total_traning_steps,
    scheduler_specific_kwargs={'min_lr': 2e-6}
)
if save_dir:
    os.makedirs(save_dir, exist_ok=True)

n_items = 0
use_task_loss = True

In [ ]:
def Matry_infonce(a, b, temperature=0.07, nested_dims=[64, 128, 256, 512, 1024]):
    """
    Matryoshka InfoNCE loss - apply contrastive loss on nested dimensions
    
    Args:
        a: [batch_size, full_dim] tensor
        b: [batch_size, full_dim] tensor  
        temperature: temperature for InfoNCE
        nested_dims: list of dimensions to apply loss on
    
    Returns:
        total_loss: sum of losses across all nested dimensions
        all_logits: dict of logits for each dimension
    """
    # Debug: check input shapes
    assert a.dim() == 2, f"Expected a to be 2D [batch, dim], got shape {a.shape}"
    assert b.dim() == 2, f"Expected b to be 2D [batch, dim], got shape {b.shape}"
    assert a.shape == b.shape, f"a and b must have same shape, got {a.shape} vs {b.shape}"
    
    total_loss = 0.0
    all_logits = {}
    
    full_dim = a.size(1)
    
    for dim in nested_dims:
        if dim > full_dim:
            print(f"Warning: skipping dim={dim} as it exceeds full_dim={full_dim}")
            continue
            
        # Slice feature dimension, not batch dimension
        q = a[:, :dim]  # [batch_size, dim]
        k = b[:, :dim]  # [batch_size, dim]
        
        # Normalize
        q = F.normalize(q, dim=-1)
        k = F.normalize(k, dim=-1)
        
        # Compute similarity matrix
        logits = torch.matmul(q, k.T) / temperature  # [batch_size, batch_size]
        
        # Labels: diagonal should match (i-th query matches i-th key)
        labels = torch.arange(q.size(0), device=q.device)
        
        # Cross entropy loss
        loss = F.cross_entropy(logits, labels)
        total_loss += loss
        
        all_logits[f'dim_{dim}'] = logits
    
    return total_loss, all_logits

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import random
from typing import List, Optional
import math

# ========== CKA Loss Implementation ==========
class CKALoss(nn.Module):
    """CKA (Centered Kernel Alignment) Loss for measuring representation similarity"""
    def __init__(self, eps=1e-8):
        super().__init__()
        self.eps = eps
    
    def forward(self, SH, TH): 
        """
        Args:
            SH: Student hidden states [batch_size * seq_len, dim_s] or [batch_size, dim_s]
            TH: Teacher hidden states [batch_size * seq_len, dim_t] or [batch_size, dim_t]
        Returns:
            CKA distance (1 - CKA similarity)
        """
        dT = TH.size(-1)
        dS = SH.size(-1)
        
        # Flatten to 2D if needed
        SH = SH.view(-1, dS).to(SH.device, torch.float64)
        TH = TH.view(-1, dT).to(SH.device, torch.float64)
        
        # Center the representations (zero mean)
        SH = SH - SH.mean(0, keepdim=True)
        TH = TH - TH.mean(0, keepdim=True)
        
        # Compute CKA similarity
        num = torch.norm(SH.t().matmul(TH), 'fro')
        den1 = torch.norm(SH.t().matmul(SH), 'fro') + self.eps
        den2 = torch.norm(TH.t().matmul(TH), 'fro') + self.eps
        
        cka_sim = num / torch.sqrt(den1 * den2)
        
        # Return distance (1 - similarity) for minimization
        return 1.0 - cka_sim


# ========== Matryoshka Hidden State Processor ==========
class MatryoshkaHiddenStateProcessor:
    """
    Self-distillation: smaller nested dims learn from largest dim
    Using CKA loss to measure representation similarity
    """
    def __init__(
        self, 
        matryoshka_dims: List[int], 
        matryoshka_weights: Optional[List[float]] = None,
        n_dims_per_step: int = -1
    ):
        # Sort dims in descending order [768, 512, 256, ...]
        self.matryoshka_dims = sorted(matryoshka_dims, reverse=True)
        
        # Default weights: smaller dims get higher weight
        if matryoshka_weights is None:
            self.matryoshka_weights = [
                1.0 / (1 + math.log(i + 1)) 
                for i in range(len(matryoshka_dims))
            ]
            #self.matryoshka_weights = [1.0 for i in range(len(matryoshka_dims))]
            
        else:
            self.matryoshka_weights = matryoshka_weights
        
        self.n_dims_per_step = n_dims_per_step
        
        # Ensure weights match dimensions
        if len(self.matryoshka_weights) != len(self.matryoshka_dims):
            raise ValueError("matryoshka_weights must have same length as matryoshka_dims")
    
    def shrink_hidden_state(self, hidden_state: torch.Tensor, dim: int) -> torch.Tensor:
        """
        Truncate hidden state to specified dimension
        Args:
            hidden_state: [batch_size, seq_len, hidden_dim] or [batch_size, hidden_dim]
            dim: target dimension
        Returns:
            Truncated tensor [..., dim]
        """
        hidden_dim = hidden_state.shape[-1]
        if dim > hidden_dim:
            raise ValueError(f"Dimension {dim} cannot be greater than hidden dimension: {hidden_dim}")
        
        # Truncate to target dimension
        truncated = hidden_state[..., :dim]
        
        return truncated
    
    def process_hidden_states(
        self, 
        hidden_state: torch.Tensor,
        cka_loss_fn: CKALoss
    ) -> torch.Tensor:
        """
        Self-distillation: Each smaller nested dim learns from the largest dim
        
        Args:
            hidden_state: [batch_size, seq_len, hidden_dim] or [batch_size, hidden_dim]
            cka_loss_fn: CKA loss function
        
        Returns:
            Combined CKA loss across all nested dimensions
        """
        # The largest dimension (teacher for self-distillation)
        largest_dim = self.matryoshka_dims[0]
        teacher_hidden = self.shrink_hidden_state(hidden_state, largest_dim)
        
        # Determine which dimensions to process (for efficiency)
        # Skip the largest dim (index 0) since it's the teacher
        dim_indices = list(range(1, len(self.matryoshka_dims)))
        
        if self.n_dims_per_step > 0 and self.n_dims_per_step < len(dim_indices):
            dim_indices = random.sample(dim_indices, self.n_dims_per_step)
            dim_indices.sort()  # Keep order for consistency
        
        total_loss = 0.0
        num_losses = 0
        
        for idx in dim_indices:
            dim = self.matryoshka_dims[idx]
            weight = self.matryoshka_weights[idx]
            
            # Check if dimension is valid
            if dim > hidden_state.shape[-1]:
                continue
            
            # Truncate to smaller dimension (student)
            student_hidden = self.shrink_hidden_state(hidden_state, dim)
            
            # Compute CKA loss between student (small dim) and teacher (large dim)
            cka_loss = cka_loss_fn(student_hidden, teacher_hidden)
            
            # Weight the loss
            weighted_loss = weight * cka_loss
            total_loss += weighted_loss
            num_losses += 1
        
        # Average the loss
        if num_losses > 0:
            total_loss = total_loss / num_losses
        
        return total_loss


# ========== Main Function: CKA Hidden State Loss ==========
def CKA_HIDDEN_STATE_LOSS(
    hidden_state1: torch.Tensor,
    hidden_state2: torch.Tensor,
    matryoshka_dims: List[int] = [1024, 512, 256, 128, 64, 32, 16],
    matryoshka_weights: Optional[List[float]] = None,
    n_dims_per_step: int = -1,
    pool_hidden: bool = True,
    attention_mask1: Optional[torch.Tensor] = None,
    attention_mask2: Optional[torch.Tensor] = None
) -> torch.Tensor:
    """
    Self-distillation with CKA loss for Matryoshka nested dimensions.
    
    Args:
        hidden_state1: [batch_size, seq_len, hidden_dim] - first view
        hidden_state2: [batch_size, seq_len, hidden_dim] - second view
        matryoshka_dims: List of nested dimensions (descending order)
        matryoshka_weights: Optional weights for each dimension
        n_dims_per_step: Number of dims to sample per step (-1 = all)
        pool_hidden: If True, mean pool token-level to sentence-level
        attention_mask1: Attention mask for first view
        attention_mask2: Attention mask for second view
    
    Returns:
        Combined CKA loss
    """
    # Initialize CKA loss and processor
    cka_loss_fn = CKALoss(eps=1e-8)
    processor = MatryoshkaHiddenStateProcessor(
        matryoshka_dims=matryoshka_dims,
        matryoshka_weights=matryoshka_weights,
        n_dims_per_step=n_dims_per_step
    )
    
    # Optional: Mean pooling to sentence-level
    if pool_hidden:
        if attention_mask1 is None or attention_mask2 is None:
            # Simple mean across sequence
            pooled1 = hidden_state1.mean(dim=1)  # [B, hidden_dim]
            pooled2 = hidden_state2.mean(dim=1)  # [B, hidden_dim]
        else:
            # Mean pooling with attention mask
            def mean_pool(hidden, mask):
                mask_expanded = mask.unsqueeze(-1).expand(hidden.size()).float()
                sum_hidden = torch.sum(hidden * mask_expanded, dim=1)
                sum_mask = torch.clamp(mask_expanded.sum(dim=1), min=1e-9)
                return sum_hidden / sum_mask
            
            pooled1 = mean_pool(hidden_state1, attention_mask1)
            pooled2 = mean_pool(hidden_state2, attention_mask2)
        
        hidden_state1 = pooled1
        hidden_state2 = pooled2
    
    # Process both views and average the loss
    loss1 = processor.process_hidden_states(hidden_state1, cka_loss_fn)
    loss2 = processor.process_hidden_states(hidden_state2, cka_loss_fn)
    
    # Average loss from both views
    total_loss = (loss1 + loss2) / 1.0
    
    return total_loss


# ========== Mean Pooling Helper ==========
def mean_pooling(model_output, attention_mask):
    """Mean pooling - take attention mask into account"""
    token_embeddings = model_output.last_hidden_state
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, dim=1)
    sum_mask = torch.clamp(input_mask_expanded.sum(dim=1), min=1e-9)
    return sum_embeddings / sum_mask


def info_nce(q, k, temperature=0.07):
    """InfoNCE contrastive loss"""
    q = F.normalize(q, dim=-1)
    k = F.normalize(k, dim=-1)
    logits = torch.matmul(q, k.T) / temperature
    labels = torch.arange(q.size(0), device=q.device)
    loss = F.cross_entropy(logits, labels)
    return loss, logits

In [ ]:
# ========== Training Loop với Token-Level CKA ==========
from torch.cuda.amp import autocast, GradScaler
import torch.optim as optim

# ========== Hyperparameters ==========
w_task = 0.7       # Task loss weight
w_cka = 0.3        # CKA self-distillation weight

matryoshka_dims = [1024, 512, 256, 128, 64, 32, 16]  # Descending order!
n_dims_per_step = -1  # -1 = use all dims, or set to 3-4 for efficiency

# ========== Training Loop ==========
for epoch in range(epochs):
    model_student.train()
    total_loss, n_items = 0.0, 0
    pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}")

    for batch in pbar:
        batch_s = {}
        for k, v in batch.items():
            if not torch.is_tensor(v):
                continue
            if k.endswith("_stu"):
                batch_s[k] = v.to(device_s, non_blocking=True)

        optimizer.zero_grad(set_to_none=True)

        with autocast(enabled=torch.cuda.is_available()):
            # ========== STUDENT Forward Pass ==========
            s_out1 = model_student(
                input_ids=batch_s["input_ids1_stu"],
                attention_mask=batch_s["attention_mask1_stu"],
                output_hidden_states=True,
                return_dict=True
            )
            s_out2 = model_student(
                input_ids=batch_s["input_ids2_stu"],
                attention_mask=batch_s["attention_mask2_stu"],
                output_hidden_states=True,
                return_dict=True
            )
            
            # Get token-level hidden states [B, L, hidden_dim]
            S_last1 = s_out1.hidden_states[-1]  # [B, L1, 768]
            S_last2 = s_out2.hidden_states[-1]  # [B, L2, 768]
            
            # Mean pooling for sentence embeddings [B, hidden_dim]
            # S_cls1 = mean_pooling(s_out1, batch_s["attention_mask1_stu"])
            # S_cls2 = mean_pooling(s_out2, batch_s["attention_mask2_stu"])

            S_cls1  = S_last1[:, 0, :]               # [B, d_s]
            S_cls2  = S_last2[:, 0, :]
            # ========== (A) Task Loss: SimCSE InfoNCE ==========
            loss_task, _ = info_nce(S_cls1, S_cls2, temperature=0.07)

            # ========== (B) CKA Hidden State Loss: Token-Level Self-Distillation ==========
            # cka_loss = CKA_HIDDEN_STATE_LOSS(
            #     hidden_state1=S_last1,  # [B, L1, 768] - token-level
            #     hidden_state2=S_last2,  # [B, L2, 768] - token-level
            #     matryoshka_dims=matryoshka_dims,
            #     n_dims_per_step=n_dims_per_step,
            #     pool_hidden=False,  # Pool tokens to sentence-level inside
            #     attention_mask1=batch_s["attention_mask1_stu"],
            #     attention_mask2=batch_s["attention_mask2_stu"]
            # )

            # ========== Combined Loss ==========
            

            nested_dims = [16, 32, 64, 128, 256, 512, 1024]  # Adjust based on your model's hidden size
            cka_loss, _ = Matry_infonce(S_cls1, S_cls2, temperature=0.07, nested_dims=nested_dims)
            loss = cka_loss
            #loss = loss_task
            loss = loss.float()

        # Backward pass
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        scheduler.step()

        # ----- Logging -----
        bs = batch_s["input_ids1_stu"].size(0)
        total_loss += loss.item() * bs
        n_items += bs

        mem_info = {}
        for dev_id in range(torch.cuda.device_count()):
            mem_alloc = torch.cuda.memory_allocated(dev_id) / 1024**2
            mem_reserved = torch.cuda.memory_reserved(dev_id) / 1024**2
            mem_info[f"gpu{dev_id}"] = f"{mem_alloc:.0f}/{mem_reserved:.0f}MB"

        avg_loss = total_loss / max(1, n_items)
        pbar.set_postfix({
            "avg_loss": f"{avg_loss:.4f}",
            "task": f"{loss_task.item():.4f}",
            "cka": f"{cka_loss.item():.4f}",
            **mem_info
        })

        # Clean up
        del S_last1, S_last2, S_cls1, S_cls2, loss, cka_loss, loss_task
        torch.cuda.empty_cache()

    # Evaluation
    print(f"\n=== Epoch {epoch+1} Evaluation ===")
    
    eval_classification_task(model_student, test_cls_tasks)
    eval_pair_task(model_student, test_pair_tasks)
    eval_sts_task(model_student, test_sts_tasks)